# Importing Libraries
import pandas as pd
import numpy as np
import os

# Importing data file

In [3]:
products = pd.read_csv(r'C:\Users\Pakshil Jani\Desktop\career foundry\Chapter-4 Python\Instacart project analysis\Data\Original Data\products.csv')

# Understanding products file

In [4]:
products.shape

(49693, 5)

In [5]:
products.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [ ]:
# From this data, we may need product_id, department_id, product_name and prices data merged with orders dataframe
# in order to analyze orders vs prices vs ordering patterns analysis.
# First we are going to perform consistency checks on products dataframe

In [18]:
products.department_id.value_counts().sort_index()

1     4007
2      548
3     1516
4     1684
5     1056
6     1139
7     4365
8      972
9     1858
10      38
11    6565
12     907
13    5371
14    1116
15    2092
16    3449
17    3085
18    1081
19    6264
20    1322
21    1258
Name: department_id, dtype: int64

In [ ]:
# All dept ids has products and hence no missing dept id. Instacart has min products offering in dept id 10 and
# highest product offering in dept id 11. Instacart is offering total 49693 products to customers

In [ ]:
# After checking all columns, we found unusual values in price column.Instacrt has products ranged between price range of
# 1 and 25. But 2 prices are unusual...14900 and 99999

In [14]:
products.prices.value_counts().sort_index()

1.0        116
1.1        296
1.2        297
1.3        286
1.4        321
          ... 
24.8         5
24.9         6
25.0         5
14900.0      1
99999.0      1
Name: prices, Length: 242, dtype: int64

In [ ]:
# Checking what these products are to figure out if prices are genuine or a possible mistake while entering data

In [22]:
products[products.prices.isin([14900,99999])]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0
33666,33664,2 % Reduced Fat Milk,84,16,99999.0


In [ ]:
# Cheese and milk don't cost that much and this looks like a data entry error. We have asked the concerned dept 
# to confirm if this data is incorrect and if so, what price can we change it to. We were advised
# to ignore these 2 items for analysis as the correct prices are not available.
# We are going to remove these 2 items from products dataframe to avoid them skewing results as they are clear outlier prices

In [25]:
products = products.drop([21554,33666],axis=0)

In [ ]:
# confirming if 14900 and 99999 values are gone

In [26]:
products[products.prices.isin([14900,99999])]

,product_id,product_name,aisle_id,department_id,prices


In [27]:
products.shape

(49691, 5)

In [ ]:
# No. of rows in the dataframe is reduced from 49693 to 49691

# Performing consistency checks on columns

In [28]:
products.describe()

,product_id,aisle_id,department_id,prices
count,49691.000000,49691.000000,49691.000000,49691.000000
mean,24844.233885,67.769113,11.728261,7.682268
std,14343.943895,38.317051,5.850337,4.200242
min,1.000000,1.000000,1.000000,1.000000
25%,12422.500000,35.000000,7.000000,4.100000
50%,24845.000000,69.000000,13.000000,7.100000
75%,37265.500000,100.000000,17.000000,11.200000
max,49688.000000,134.000000,21.000000,25.000000


In [ ]:
# Value counts for all columns are same
# Min and max values look logical. Since we dropped 2 products due to their incorrect prices, product_id
# column has max value 49688 instead of 49690

# Checking data types of columns

In [30]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49691 entries, 0 to 49692
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     49691 non-null  int64  
 1   product_name   49675 non-null  object 
 2   aisle_id       49691 non-null  int64  
 3   department_id  49691 non-null  int64  
 4   prices         49691 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 2.3+ MB


# Checking for any mixed data type in any column

In [31]:
for col in products.columns.tolist():
  weird = (products[[col]].applymap(type) != products[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (products[weird]) > 0:
    print (col)

product_name


In [59]:
products.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [ ]:
# product_name has 16 rows with null values and that may be the reason query showing mixed data type
# Let us find out these rows

In [64]:
products[products.product_name.isnull()]

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [ ]:
# Checking for product_id 14 to see if we can make sense of this missing value

In [66]:
products[products.department_id == 14]

,product_id,product_name,aisle_id,department_id,prices
27,28,Wheat Chex Cereal,121,14,10.1
33,34,NaN,121,14,12.2
67,68,"Pancake Mix, Buttermilk",130,14,13.7
89,90,Smorz Cereal,121,14,3.9
210,211,Gluten Free Organic Cereal Coconut Maple Vanilla,130,14,3.6
...,...,...,...,...,...
49330,49326,Cereal Variety Fun Pack,121,14,9.1
49395,49391,Light and Fluffy Buttermilk Pancake Mix,130,14,2.0
49547,49543,Chocolate Cheerios Cereal,121,14,10.8
49637,49633,Shake 'N Pour Buttermilk Pancake Mix,130,14,14.2


In [ ]:
# These missing values are string objects and we can't impute them like numeric values using mean or median. 
#We have to go back to the data source and ask for the missing product names. For the purpose of this project
# our tutor advised us to ignore these 16 rows for the purpose of our analysis

In [ ]:
# Creating new dataframe products_clean which does not include any missing values

In [81]:
products_clean = products[products.product_name.isnull()==False]

In [82]:
products_clean.shape

(49675, 5)

In [ ]:
# 49691-16= 49675 This looks fine. Checking again for any missing values after correction

In [83]:
products_clean.isnull().sum()

product_id       0
product_name     0
aisle_id         0
department_id    0
prices           0
dtype: int64

# Checking for duplicate values

In [85]:
products_clean[products_clean.duplicated()]

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [ ]:
# Before we do anything to these duplicates, we want to see them in dataframe to ensure that they are
# genuine duplicates

In [89]:
products_clean[products_clean.product_id.isin([462,18458,26808,35306,35491])]

,product_id,product_name,aisle_id,department_id,prices
461,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18458,18458,Ranger IPA,27,5,9.2
18459,18458,Ranger IPA,27,5,9.2
26809,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35308,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35494,35491,Adore Forever Body Wash,127,11,9.9
35495,35491,Adore Forever Body Wash,127,11,9.9


In [ ]:
# These product ids are genuine duplicates. Now we can remove them from the database and name a new database

In [ ]:
# Before we drop these duplicate values, we are going to export this csv file. In case, if in future, we need to revert back
# to it, we can do that.

In [90]:
products_clean.to_csv(r'C:\Users\Pakshil Jani\Desktop\career foundry\Chapter-4 Python\Instacart project analysis\Data\Prepared Data\products_clean.csv')

In [ ]:
# Now creating new dataframe by dropping missing values

In [91]:
products_clean_duplicate = products_clean.drop_duplicates()

In [92]:
products_clean_duplicate.shape

(49670, 5)

In [ ]:
# After dropping 5 duplicates, 49675-5=49670 rows remain
# We want to confirm that only duplicates have been dropped. Original rows remain in the database.

In [94]:
products_clean_duplicate[products_clean_duplicate.product_id.isin([462,18458,26808,35306,35491])]

,product_id,product_name,aisle_id,department_id,prices
461,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18458,18458,Ranger IPA,27,5,9.2
26809,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35308,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35494,35491,Adore Forever Body Wash,127,11,9.9


In [ ]:
# Original 5 product ids are still there so we are good to proceed to next step

# Since all checks are done, we are going to export this clean dataframe as csv file

In [95]:
products_clean_duplicate.to_csv(r'C:\Users\Pakshil Jani\Desktop\career foundry\Chapter-4 Python\Instacart project analysis\Data\Prepared Data\products_all_clean.csv')

In [96]:
for col in products_clean_duplicate.columns.tolist():
  weird = (products_clean_duplicate[[col]].applymap(type) != products_clean_duplicate[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (products_clean_duplicate[weird]) > 0:
    print (col)

In [ ]:
# In the earlier analysis, product_name column has mixed data type so checked again and now no mixed data type

In [ ]:
# In products dataframe, department_id is there as a column but department names are not provided. So we asked if
# we can be provided with department names so we can include them in a separate column and can be understood better

# Importing data file of the new department dataframe provided to us

In [98]:
departments = pd.read_csv(r'C:\Users\Pakshil Jani\Desktop\career foundry\Chapter-4 Python\Instacart project analysis\Data\Original Data\departments.csv')

# Checking this dataframe details

In [99]:
departments.shape

(1, 22)

In [101]:
departments.head(22)

,department_id,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,department,frozen,other,bakery,produce,alcohol,international,beverages,pets,dry goods pasta,...,meat seafood,pantry,breakfast,canned goods,dairy eggs,household,babies,snacks,deli,missing


In [ ]:
# For better visibility, we are want to see department_ids in a column and department names in a row

In [113]:
df = departments.T

In [114]:
df

,0
department_id,department
1,frozen
2,other
3,bakery
4,produce
5,alcohol
6,international
7,beverages
8,pets
9,dry goods pasta


In [ ]:
# Performing index_rest function to get header

In [115]:
df.reset_index()

,index,0
0,department_id,department
1,1,frozen
2,2,other
3,3,bakery
4,4,produce
5,5,alcohol
6,6,international
7,7,beverages
8,8,pets
9,9,dry goods pasta


In [ ]:
# We want to see first row as header

In [130]:
new_header=df.iloc[0]

In [ ]:
# Everything from 1st row onwards to be our new cleaned dataframe

In [133]:
departments_clean=df[1:]

In [134]:
departments_clean

department_id,department
1,frozen
2,other
3,bakery
4,produce
5,alcohol
6,international
7,beverages
8,pets
9,dry goods pasta
10,bulk


# Exporting this dataset as csv file

In [136]:
departments_clean.to_csv(r'C:\Users\Pakshil Jani\Desktop\career foundry\Chapter-4 Python\Instacart project analysis\Data\Prepared Data\dept_clean.csv')